# iLQR
Location for Linear-Quadratic Regulator (LQR) and iLQR for interactive trajectory planning.

References:
- [1] - [Dewolf - studywolf_control](https://github.com/studywolf/control/blob/master/studywolf_control/controllers/ilqr.py)
- [2] - [Fridovich-Keil - ilqgames](https://github.com/HJReachability/ilqgames/blob/master/python/dynamical_system.py)
- [3] - [Jackson, Howell - iLQR Tutorial](http://roboticexplorationlab.org/papers/iLQR_Tutorial.pdf)
- [4] - [Jackson - AL iLQR Tutorial](https://bjack205.github.io/papers/AL_iLQR_Tutorial.pdf)
- [5] - [Sears-Collins - Linear Quadratic Regulator Python Example](https://automaticaddison.com/linear-quadratic-regulator-lqr-with-python-code-example/)

In [1]:
import sys
sys.path.append('/mnt/c/Users/ZachJW/Documents/Grad/Research/icon/src/')

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are

from dynamics import CarDynamics, UnicycleDynamics, BicycleDynamics
from dynamics import DoubleInt1dDynamics, DoubleInt2dDynamics
from cost import QuadraticCost

%matplotlib ipympl
np.set_printoptions(precision=3,suppress=True)

In [2]:
plt.figure()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

### iLQR

In [16]:
class iLQR(object):
    """iLQR solver"""
    
    # -- Forward Pass --
    GAMMA = 0.5 # scaling factor
    ALPHA_MIN = 1e-6 # scaling bounds
    BETA_RANGE = (1e-4, np.inf) # acceptable cost reduction ratio
    
    # -- Backard Pass
    PHI = 2.0 # regularization scaling
    RHO_MAX = 1e3 # regularization ceiling
    
    # -- iLQR --
    N_LQR_ITER = 20 # number of successful iLQR iterations
    N_LS_ITER = 20 # number of line search iterations
    EPSILON = 1e-3 # convergence criterion
    J_MAX = 1e8 # cost ceiling for evaluating forward pass
    
    def __init__(self, dynamics, cost, N=10):
        assert N > 1
        
        self.dynamics = dynamics
        self.cost = cost
        self.cost_hist = None
        self.N = N
    
    @property
    def n_x(self):
        return self.dynamics.n_x
    
    @property
    def n_u(self):
        return self.dynamics.n_u
    
    @property
    def dt(self):
        return self.dynamics.dt
    
    @property
    def xf(self):
        return self.cost.xf
    
    @staticmethod
    def expected_redu(ΔV, α):
        """Compute the expected reduction in cost (49) [4]."""
        return np.sum(α*ΔV[0,:] + α**2 * ΔV[1,:])
    
    def rollout(self, x0, U):
        """Rollout the system from an initial state with a control sequence U."""
        
        X = np.zeros((self.N+1, self.n_x))
        X[0] = x0
        J = 0.0
        
        for t in range(self.N):
            X[t+1] = self.dynamics(X[t], U[t])
            J += self.cost(X[t], U[t])
        J += self.cost(X[-1], np.zeros(self.n_u), terminal=True)

        return X, J
    
    def line_search(self, J, ΔV, α):
        """Determine whether to accept the updated trajectory and controls."""
        
        # Compare actual to anticipated cost reduction.
        ΔJ = self.cost_hist[-1] - J
        expected_ΔV = self.expected_redu(ΔV, α)
        z = ΔJ / -expected_ΔV

        # Reduction in cost means success! Accept new trajectories and reduce the 
        # regularization to converge slower.
        if ΔJ >= 0:
        # if self.BETA_RANGE[0] <= z and z <= self.BETA_RANGE[1]:
            self.cost_hist.append(J)
            accept = True

        # Increase in cost means failure... Reject new trajectories and increase the
        # regularization to converge quicker.
        else:
            accept = False
            α = max(α*self.GAMMA, self.ALPHA_MIN)
        
        return accept, α, ΔJ, z
    
    def forward_pass(self, X, U, K, d, ΔV):
        """Forward pass based on algorithm 2 [4]."""
        
        # NOTE: literature says 1.0, but this gives a really poor initial forward pass.
        α = 1.0

        # Conduct a line search to find a satisfactory trajectory where we continually
        # decrease α. We're effectively getting closer to the linear approximation in
        # the LQR case.
        for i_ls in range(self.N_LS_ITER):
            X_next = np.zeros((self.N+1, self.n_x))
            U_next = np.zeros((self.N, self.n_u))

            X_next[0] = X[0]
            J = 0.0

            for t in range(self.N):
                δx = X_next[t] - X[t]
                δu = K[t]@δx + α*d[t]
                U_next[t] = U[t] + δu

                X_next[t+1] = self.dynamics(X_next[t], U_next[t])
                J += self.cost(X_next[t], U_next[t])
            J += self.cost(X_next[-1], None, terminal=True)
            
            accept, α, ΔJ, z = self.line_search(J, ΔV, α)
            print(f'\t[FP] {i_ls+1}/{self.N_LS_ITER}\t{J = :.6g}\t{α = :.6g}\t{z = :.6g}') 
            # break # DBG
            if accept:
                break
        
        return X_next, U_next, J, ΔJ, accept
    
    def backward_pass(self, X, U, ρ=0.0):
        """Backward pass based on algorithm 1 [4]."""
        
        # ρ = 0.0
        K = np.zeros((self.N, self.n_u, self.n_x)) # linear feedback gain
        d = np.zeros((self.N, self.n_u)) # feedforward gain
        ΔV = np.zeros((self.N, 2)) # two coefficients in expected cost reduction
        
        # Initialize cost jacobian and hessian. NOTE: U isn't actually used in the quadraticization
        # aside from its dimensions, so we just give it the right shape.
        L_x, _, L_xx, _, _ = self.cost.quadraticize(X[-1], np.zeros(self.n_u), terminal=True)
        p = L_x
        P = L_xx

        for t in range(self.N-1, -1, -1):
            L_x, L_u, L_xx, L_uu, L_ux = self.cost.quadraticize(X[t], U[t])
            A, B = self.dynamics.linearize(X[t], U[t])
            
            # Another small difference with pydrake dynamics
            L_uu *= self.dt**2
            
            Q_x = L_x + A.T @ p
            Q_u = L_u + B.T @ p
            Q_xx = L_xx + A.T @ P @ A
            Q_uu = L_uu + B.T @ P @ B
            Q_ux = L_ux + B.T @ P @ A
            Q_xu = L_ux.T + A.T @ P @ B
            
            Q_uu += ρ * np.eye(self.n_u)
            Q_uu_inv = np.linalg.inv(Q_uu)
            
            K[t] = -Q_uu_inv @ Q_ux
            d[t] = -Q_uu_inv @ Q_u
            ΔV[t] = np.array([[d[t].T @ Q_u, 0.5*d[t].T @ Q_uu @ d[t]]])
            
            p = Q_x + K[t].T @ Q_uu @ d[t] + K[t].T @ Q_u + Q_xu @ d[t]
            P = Q_xx + K[t].T @ Q_uu @ K[t] + K[t].T @ Q_ux + Q_xu @ K[t]
            
        return K, d, ΔV
    
    def is_verged(self, ΔJ):
        """Determine whether we've solved the OCP to within a threshold or diverged."""
        
        if abs(ΔJ) < self.EPSILON:
            print('Convergence!')
            return True
        elif self.cost_hist[-1] > self.J_MAX:
            print('Divergence...')
            return True
        return False
        
    def run(self, x0, n_lqr_iter=None, ρ0=1.0):
        """Solve the OCP."""
        
        ρ = ρ0
        if n_lqr_iter is None:
            n_lqr_iter = self.N_LQR_ITER
        
        U = np.random.randn(self.N, self.n_u) * 1e-3
        X, J = self.rollout(x0, U)
        
        self.cost_hist = [J]
        print(f'[run] initial: {J = :.6g}')        
        
        for i in range(n_lqr_iter):
            # Backward recurse to compute gain matrices.
            K, d, ΔV = self.backward_pass(X, U, ρ)

            # Rollout the full trajectory.
            X_next, U_next, J, ΔJ, accept = self.forward_pass(X, U, K, d, ΔV)
            
            # Successful iteration, accept the candidate trajectories.
            if accept and J < self.J_MAX:
                # print('[run] Accepted new trajectory!')
                X = X_next
                U = U_next
                # ρ = 0.0
                if ρ < 1e-12:
                    ρ = 1.0
                ρ /= self.PHI

            # Forward pass line search failed. Retry backward pass with more regularization.
            else:
                # TEMP: Regularization is pointless for quadratic costs.
                print('[run] Failed line search.. giving up.')
                break
                
                print('[run] Failed line search.. increasing ρ.')
                ρ = max(ρ*self.PHI, 1.0)
                
                # Give up if we've exceeded the regularization ceiling.
                if ρ > self.RHO_MAX:
                    print('[run] Failed regularization.. giving up.')
                    break
            
            print(f'[run] {i+1}/{n_lqr_iter}\tρ: {ρ:.3g}\t{ΔJ = :.6g}')
            if accept and self.is_verged(ΔJ):
                break

        return X, U, J

In [27]:
# %%prun
dt = 0.01
N = 20
n_lqr_iter = 50
ρ0 = 0.0

# dynamics = DoubleInt1dDynamics(dt)
# x0 = np.array([2, 0])
# xf = np.array([0, 1])
# Q = np.diag([1, 1])
# R = np.eye(1) * 0.01

dynamics = DoubleInt2dDynamics(dt)
x0 = np.array([1, 1, 10, 0])
xf = np.array([0, 0, 0, 0])
Q = np.diag([1, 1, .1, .1])
R = np.eye(2) * 0.01

# dynamics = UnicycleDynamics(dt)
# x0 = np.array([2, -1, 0, np.pi/2])
# xf = np.array([0, 0, 1, 0])
# Q = np.diag([1, 1, 1, 1])
# R = np.eye(2) * 0.1

# dynamics = BicycleDynamics(dt)
# x0 = np.array([-1, 1, np.pi/2, -1, 0])
# xf = np.array([0, 0, 0, 0, 0])
# Q = np.diag([1, 1, 1, 1, 1])
# R = np.eye(2) * 0.01

term_penalty = 1e5
quad_cost = QuadraticCost(xf, Q, R, term_penalty)
ilqr = iLQR(dynamics, quad_cost, N=N)
X, U, cost = ilqr.run(x0, n_lqr_iter)
dynamics.plot(X, quad_cost.xf, False)

[run] initial: J = 2.00031e+06
	[FP] 1/20	J = 94348.3	α = 1	z = 30.8152
[run] 1/50	ρ: 0.5	ΔJ = 1.90596e+06
	[FP] 1/20	J = 14854.6	α = 1	z = 13.6447
[run] 2/50	ρ: 0.25	ΔJ = 79493.7
	[FP] 1/20	J = 8256.21	α = 1	z = 20.3785
[run] 3/50	ρ: 0.125	ΔJ = 6598.36
	[FP] 1/20	J = 8088.44	α = 1	z = 30.1439
[run] 4/50	ρ: 0.0625	ΔJ = 167.774
	[FP] 1/20	J = 8087.31	α = 1	z = 11.8041
[run] 5/50	ρ: 0.0312	ΔJ = 1.13065
	[FP] 1/20	J = 8087.24	α = 1	z = 1.95979
[run] 6/50	ρ: 0.0156	ΔJ = 0.0623113
	[FP] 1/20	J = 8087.24	α = 1	z = 1.0539
[run] 7/50	ρ: 0.00781	ΔJ = 0.00831651
	[FP] 1/20	J = 8087.24	α = 0.5	z = -0.773096
	[FP] 2/20	J = 8087.24	α = 0.5	z = 1.29682
[run] 8/50	ρ: 0.00391	ΔJ = 0.000643756
Convergence!


In [9]:
U = np.random.randn(N, 2) * 1e-6
x0 = np.array([-1, -1, np.pi/3, 1, 0])
X, cost = ilqr.rollout(x0, U)

ValueError: could not broadcast input array from shape (5,) into shape (4,)

In [79]:
K = np.zeros((ilqr.N, ilqr.n_u, ilqr.n_x)) # linear feedback gain
d = np.zeros((ilqr.N, ilqr.n_u)) # feedforward gain
ΔV = np.zeros((ilqr.N, 2)) # two coefficients in expected cost reduction
K, d, ΔV = ilqr.backward_pass(X, U, 0.0)
X_next, U_next, J, ΔJ, accept = ilqr.forward_pass(X, U, K, d, ΔV)
X_next

(21, 5) (20, 2)
	[FP] 1/20	J = 75.4262	α = 0.5	z = -17.0487


array([[-1.   , -1.   ,  1.047,  1.   ,  0.   ],
       [-0.95 , -0.913,  1.047,  1.06 , -0.541],
       [-0.897, -0.822,  0.984,  1.015, -0.686],
       [-0.841, -0.737,  0.901,  0.935, -0.678],
       [-0.783, -0.664,  0.825,  0.849, -0.615],
       [-0.725, -0.601,  0.765,  0.764, -0.544],
       [-0.67 , -0.549,  0.719,  0.686, -0.485],
       [-0.618, -0.503,  0.683,  0.614, -0.439],
       [-0.571, -0.465,  0.654,  0.548, -0.404],
       [-0.527, -0.431,  0.631,  0.488, -0.377],
       [-0.488, -0.402,  0.611,  0.432, -0.353],
       [-0.452, -0.378,  0.595,  0.38 , -0.331],
       [-0.421, -0.356,  0.582,  0.331, -0.308],
       [-0.393, -0.338,  0.572,  0.285, -0.284],
       [-0.369, -0.323,  0.563,  0.241, -0.256],
       [-0.349, -0.31 ,  0.557,  0.199, -0.224],
       [-0.332, -0.299,  0.553,  0.158, -0.188],
       [-0.318, -0.291,  0.55 ,  0.119, -0.147],
       [-0.308, -0.285,  0.548,  0.082, -0.105],
       [-0.301, -0.281,  0.547,  0.048, -0.063],
       [-0.297, -0.2

In [ ]:
L_x, _, L_xx, _, _ = ilqr.cost.quadraticize(X[-1], np.zeros(ilqr.n_u), terminal=True)
P = L_xx
p = L_x
t = N
ρ = 0.0

In [76]:
t -= 1
L_x, L_u, L_xx, L_uu, L_ux = ilqr.cost.quadraticize(X[t], U[t])
A, B = ilqr.dynamics.linearize(X[t], U[t])

Q_x = L_x + A.T @ p
Q_u = L_u + B.T @ p
Q_xx = L_xx + A.T @ P @ A
Q_uu = L_uu*dt**2 + B.T @ P @ B
Q_ux = L_ux + B.T @ P @ A
Q_xu = L_ux.T + A.T @ P @ B

Q_uu += ρ * np.eye(ilqr.n_u)
Q_uu_inv = np.linalg.inv(Q_uu)

K[t] = -Q_uu_inv @ Q_ux
d[t] = -Q_uu_inv @ Q_u
ΔV[t] = np.array([[d[t].T @ Q_u, 0.5*d[t].T @ Q_uu @ d[t]]])

p = Q_x + K[t].T @ Q_uu @ d[t] + K[t].T @ Q_u + Q_xu @ d[t]
P = Q_xx + K[t].T @ Q_uu @ K[t] + K[t].T @ Q_ux + Q_xu @ K[t]

p, P

(array([-0.362,  3.765,  6.494,  3.467,  0.509]),
 array([[ 5.998, -0.003, -0.52 ,  0.239, -0.017],
        [-0.003,  5.994,  0.3  ,  0.415,  0.01 ],
        [-0.52 ,  0.3  ,  6.092, -0.   ,  0.483],
        [ 0.239,  0.415, -0.   ,  3.259, -0.   ],
        [-0.017,  0.01 ,  0.483, -0.   ,  3.259]]))

### LQR

In [8]:
class LQR(iLQR):
    """Linear Quadratic Regulator that assumes linear dynamics and quadratic costs."""
    
    def __init__(self, dynamics, cost, N=10, Q=None, R=None):
        self.cost = cost
        self.dynamics = dynamics
        self.N = N
        
        if Q is None:
            Q = np.eye(self.n_x)
        if R is None:
            R = np.eye(self.n_u)
        self.Q = Q
        self.R = R
        
    def backward_pass(self, X, U):
        """Compute the optimal feedforward gain K."""
        
        K = np.zeros((self.N, self.n_u, self.n_x))
        P = np.zeros((self.N, self.n_x, self.n_x))
        P[-1] = self.Q
        
        for t in range(self.N-2, -1, -1):
            A, B = self.dynamics.linearize(X[t], U[t]) 
            # Feedforward gain [4] (30)
            K[t] = np.linalg.inv(self.R + B.T @ P[t+1] @ B) @ B.T @ P[t+1] @ A
            # Cost-to-go [4] (32)
            P[t] = self.Q + (A.T @ P[t+1] @ A) - A.T @ P[t+1].T @ B @ K[t]
            
            # Q: What's the difference with these two?
            # P[t] = la.solve_discrete_are(A, B, Q, R)
            # K[t] = np.linalg.pinv(R + B.T @ P[t] @ B) @ B.T @ P[t] @ A

        return K
    
    def forward_pass(self, X, U, K):
        """Apply the feedforward gain to compute the next trajectory."""
        
        X_next = np.zeros((self.N, self.n_x))
        U_next = np.zeros((self.N, self.n_u))
        
        X_next[0] = x0
        J = 0.0

        for t in range(self.N-1):
            U_next[t] = -K[t] @ (X_next[t] - self.xf)
            X_next[t+1] = self.dynamics(X_next[t], U_next[t])
            J += self.cost(X_next[t], U_next[t])
        J += self.cost(X_next[-1], U_next[-1], terminal=True)

        return X_next, U_next, J
    
    def run(self, x0, n_iter=10):
        """Solve the LQR OCP."""
        
        U = np.random.randn(self.N, self.n_u) * 1e-3
        X, J0 = self.rollout(x0, U)

        K = self.backward_pass(X, U)
        X, U, Jf = self.forward_pass(X, U, K)
        print(f'J0: {J0:.3g}\tJf: {Jf:.3g}')

        return X, U, Jf

In [10]:
N = 20
dt = 1.0

dynamics = DoubleInt1dDynamics(dt)
x0 = np.array([10, 0])
xf = np.array([0, 0])
Q = np.diag([1, 1])
R = np.eye(1)

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([1, 1, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.eye(4)
# R = np.eye(2)

# dynamics = CarDynamics(dt)
# x0 = np.array([5.0, 5.0, 0])
# xf = np.array([0, 0, 0])
# Q = np.diag([2.0, 2.0, 1.0])
# R = np.eye(2)

# dynamics = UnicycleDynamics(dt)
# x0 = np.array([-1, -1, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1.0, 1.0, 1.0, 1.0])
# R = np.eye(2)

quad_cost = QuadraticCost(xf, Q, R)
lqr = LQR(dynamics, quad_cost, N=N, Q=Q)
X, U, J = lqr.run(x0)
dynamics.plot(X, xf)

J0: 1e+05	Jf: 237
